In [10]:
map_editor_path = './temp/map_editor_messages.csv'
mission_editor_path = './temp/mission_editor_messages.csv'
flight_school_path = './temp/flight_school_messages.csv'

In [11]:
import pandas as pd
import datetime
import json

In [12]:
def generate_data(input_df, QA_dataset, title):
    length = input_df.shape[0]
    for index, row in input_df.iterrows():
        row_str = row['Content']
        if '?' in row_str and len(row_str.split(' ')) > 4:   
            context = ''

            for idx in range(index, index+3):
                if idx < length:
                    context += input_df.iloc[idx]['Content'] + "\n"
            context += "\n"

            QA_dataset['data'].append(
                {
                    "title": title,
                    "paragraphs": [
                        {
                            "qas":[
                                {   
                                    "question": row_str,
                                    "id": index,
                                    "answers":[
                                        {
                                            "text": row_str,
                                            "answer_start": 0
                                        }
                                    ]
                                }
                            ],
                            "context": context
                        }
                    ]
                }
            )

In [13]:
def clean_df(input_df):
    input_df['Date'] = pd.to_datetime(input_df['Date'], format="%m/%d/%Y %I:%M %p")
    input_df = input_df[input_df['Content'].isna() == False]
    input_df = input_df[input_df['Date'] >= datetime.datetime.strptime("2021", "%Y")]
    input_df = input_df.reset_index()
    return input_df

In [14]:
mission_editor_df = pd.read_csv(mission_editor_path, header=0)
map_editor_df = pd.read_csv(mission_editor_path, header=0)
flight_school_df = pd.read_csv(mission_editor_path, header=0)

In [15]:
mission_editor_df = clean_df(mission_editor_df)
map_editor_df = clean_df(map_editor_df)
flight_school_df = clean_df(flight_school_df)

In [16]:
QA_dataset = {
    "version": "v1.0",
    "data": []
}


In [17]:
generate_data(mission_editor_df, QA_dataset, "mission_editor")
print(len(QA_dataset['data']))
generate_data(map_editor_df, QA_dataset, "map_editor")
print(len(QA_dataset['data']))
generate_data(flight_school_df, QA_dataset, "flight_school")
print(len(QA_dataset['data']))

3909
7818
11727


In [18]:
open('./training/QA_training_data.json','w').write(json.dumps(QA_dataset))

7482680